Install library for QC

In [252]:
%pip install dwave-ocean-sdk
%pip install dwave-neal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Подключим все необходимые библиотеки

In [253]:
import random
import math
import itertools

import dimod
from dimod import BinaryQuadraticModel
from dimod.reference.samplers import ExactSolver
from neal import SimulatedAnnealingSampler

Сгенерируем случайные данные

In [254]:
# ClientsNum = random.randint(1, 3)
ClientsNum = 6
DepotsNum = random.randint(1, 1)
CarsNum = random.randint(DepotsNum, DepotsNum)

Distances = [[0 for _ in range(ClientsNum + DepotsNum)] for _ in range(ClientsNum + DepotsNum)]
for i in range(ClientsNum + DepotsNum):
    for j in range(i + 1, ClientsNum + DepotsNum):
        Distances[i][j] = random.randint(100, 5000)
        Distances[j][i] = Distances[i][j]

# GoodsDepot = [random.randint(50, 100) for _ in range(DepotsNum)]
# GoodsCar = [random.randint(50, 100) for _ in range(CarsNum)]
# GoodsClients = [random.randint(1, 9) for _ in range(ClientsNum)]

CarDepot = [[0 for _ in range(DepotsNum)] for _ in range(CarsNum)]
CarDepotMap = {}
for i in range(CarsNum):
    depot = random.randint(0, DepotsNum - 1)
    CarDepot[i][depot] = 1
    CarDepotMap[i] = depot

# Optimized parameters amount

xNum = ClientsNum * ClientsNum * CarsNum
uNum = ClientsNum * CarsNum
nNum = ClientsNum * CarsNum

print("Total Amount of Parameters: ", xNum + uNum + nNum)

Total Amount of Parameters:  48


Инициализируем матрицу QUBO

In [255]:
BigConstant = 1000000

h = {}
J = {}

for i in range(ClientsNum):
    for j in range(ClientsNum):
        for k in range(CarsNum):
            h[f"x{i}{j}{k}"] = 0

for i in range(ClientsNum):
    for k in range(CarsNum):
        h[f"u{i}{k}"] = 0

for i in range(ClientsNum):
    for k in range(CarsNum):
        h[f"n{i}{k}"] = 0

keys = list(h.keys())

for i in range(len(keys)):
    for j in range(len(keys)):
        if i != j:
            J[(keys[i], keys[j])] = 0

print(h, J)

{'x000': 0, 'x010': 0, 'x020': 0, 'x030': 0, 'x040': 0, 'x050': 0, 'x100': 0, 'x110': 0, 'x120': 0, 'x130': 0, 'x140': 0, 'x150': 0, 'x200': 0, 'x210': 0, 'x220': 0, 'x230': 0, 'x240': 0, 'x250': 0, 'x300': 0, 'x310': 0, 'x320': 0, 'x330': 0, 'x340': 0, 'x350': 0, 'x400': 0, 'x410': 0, 'x420': 0, 'x430': 0, 'x440': 0, 'x450': 0, 'x500': 0, 'x510': 0, 'x520': 0, 'x530': 0, 'x540': 0, 'x550': 0, 'u00': 0, 'u10': 0, 'u20': 0, 'u30': 0, 'u40': 0, 'u50': 0, 'n00': 0, 'n10': 0, 'n20': 0, 'n30': 0, 'n40': 0, 'n50': 0} {('x000', 'x010'): 0, ('x000', 'x020'): 0, ('x000', 'x030'): 0, ('x000', 'x040'): 0, ('x000', 'x050'): 0, ('x000', 'x100'): 0, ('x000', 'x110'): 0, ('x000', 'x120'): 0, ('x000', 'x130'): 0, ('x000', 'x140'): 0, ('x000', 'x150'): 0, ('x000', 'x200'): 0, ('x000', 'x210'): 0, ('x000', 'x220'): 0, ('x000', 'x230'): 0, ('x000', 'x240'): 0, ('x000', 'x250'): 0, ('x000', 'x300'): 0, ('x000', 'x310'): 0, ('x000', 'x320'): 0, ('x000', 'x330'): 0, ('x000', 'x340'): 0, ('x000', 'x350'): 0,

Заполним матрицу QUBO исходя из ограничений и минимизируемого функционала

Напишем функцию для раскрытия выражения вида $(a_1 + a_2 + ... + a_n)^2$, потому что почти все ограничения к этому сводятся

In [256]:
def square(vals): # data example: [(1, "x1"), (-2, "x3"), (3, "x4")]

    for val in vals:
        if val[1] != "":
            h[val[1]] += val[0] * val[0]
        
    for v1 in vals:
        for v2 in vals:
            if v1[1] == v2[1]:
                continue
            if v1[1] == "" and v2[1] == "":
                continue

            if v1[1] != "" and v2[1] != "":
                if (v1[1], v2[1]) not in J.keys():
                    J[(v1[1], v2[1])] = 0
                J[(v1[1], v2[1])] += v1[0] * v2[0]
            else:
                h[v1[1]+v2[1]] += v1[0] * v2[0]

_Целевая функция:_ Функия описывающая издержки для передвижения всех машин. Данную функцию мы хотим минимизировать.

In [257]:
for k in range(CarsNum):
    for i in range(ClientsNum):
        for j in range(ClientsNum):
            print(f"x{i}{j}{k} += {Distances[i][j]}")
            h[f"x{i}{j}{k}"] += Distances[i][j]

for k in range(CarsNum):
    for i in range (ClientsNum):
        for d in range(DepotsNum):
            print(f"u{i}{k} += {Distances[i][ClientsNum + d]} * {CarDepot[k][d]}")
            h[f"u{i}{k}"] += Distances[i][ClientsNum + d] * CarDepot[k][d]

for k in range(CarsNum):
    for i in range (ClientsNum):
        for d in range(DepotsNum):
            print(f"n{i}{k} += {Distances[i][ClientsNum + d]} * {CarDepot[k][d]}")
            h[f"n{i}{k}"] += Distances[i][ClientsNum + d] * CarDepot[k][d]

x000 += 0
x010 += 4757
x020 += 4134
x030 += 4512
x040 += 2745
x050 += 303
x100 += 4757
x110 += 0
x120 += 2666
x130 += 3120
x140 += 1103
x150 += 160
x200 += 4134
x210 += 2666
x220 += 0
x230 += 387
x240 += 3985
x250 += 2584
x300 += 4512
x310 += 3120
x320 += 387
x330 += 0
x340 += 4268
x350 += 2130
x400 += 2745
x410 += 1103
x420 += 3985
x430 += 4268
x440 += 0
x450 += 2261
x500 += 303
x510 += 160
x520 += 2584
x530 += 2130
x540 += 2261
x550 += 0
u00 += 1213 * 1
u10 += 761 * 1
u20 += 4083 * 1
u30 += 4007 * 1
u40 += 2668 * 1
u50 += 3089 * 1
n00 += 1213 * 1
n10 += 761 * 1
n20 += 4083 * 1
n30 += 4007 * 1
n40 += 2668 * 1
n50 += 3089 * 1


_Ограничение 1:_ Нельзя поехать к тому же покупателю.

In [258]:
for i in range(ClientsNum):
    for k in range(CarsNum):
        print(f"x{i}{i}{k} += {BigConstant}")
        h[f"x{i}{i}{k}"] += BigConstant

x000 += 1000000
x110 += 1000000
x220 += 1000000
x330 += 1000000
x440 += 1000000
x550 += 1000000


_Ограничение 2:_ К каждому покупателю должна приехать ровно одна машина.

In [259]:
for i in range(ClientsNum):

    vals = []

    vals.append((BigConstant, ""))

    for k in range(CarsNum):
        for j in range(ClientsNum):
            vals.append((-BigConstant, f"x{j}{i}{k}"))
        vals.append((-BigConstant, f"u{i}{k}"))

    square(vals)

_Ограничение 3:_ От каждого покупателя должна уехать ровно одна машина.

In [260]:
for i in range(ClientsNum):

    vals = []

    vals.append((BigConstant, ""))

    for k in range(CarsNum):
        for j in range(ClientsNum):
            vals.append((-BigConstant, f"x{i}{j}{k}"))
        vals.append((-BigConstant, f"n{i}{k}"))

    square(vals)

_Ограничение 4:_ Каждая машина должна отъехать ровно с одного склада.

In [261]:
for k in range(CarsNum):

    vals = []

    vals.append((BigConstant, ""))

    for i in range(ClientsNum):
        vals.append((-BigConstant, f"u{i}{k}"))

    square(vals)

_Ограничение 5:_ Каждая машина должна приехать ровно на один склад.

In [262]:
for k in range(CarsNum):

    vals = []

    vals.append((BigConstant, ""))

    for i in range(ClientsNum):
        vals.append((-BigConstant, f"n{i}{k}"))

    square(vals)

_Ограничение 6:_ "Целостность маршрута". Если машина приехала к покупателю, то она должна уехать от него.

In [263]:
for i in range(ClientsNum):
    for k in range(CarsNum):
        
        vals = []

        vals.append((BigConstant, f"u{i}{k}"))
        vals.append((-BigConstant, f"n{i}{k}"))

        for j in range(ClientsNum):
            vals.append((BigConstant, f"x{j}{i}{k}"))
            vals.append((-BigConstant, f"x{i}{j}{k}"))

        square(vals)

_Ограничение 7:_ У машин не должно быть возможности иметь на своем маршруте цикл из городов, который они не посещают.

In [264]:
def get_subsets(n):
    res = []
    for i in range(1 << n):
        cur = []
        for j in range(n):
            if (i & (1 << j)):
                cur.append(j)
        res.append(cur)
    return res

id = 0

for S in get_subsets(ClientsNum):
    if len(S) >= 2:

        vals = []

        vals.append((BigConstant * (-len(S) + 1), ""))

        for k in range(CarsNum):
            for i in S:
                for j in S:
                    if i != j:
                        vals.append((BigConstant, f"x{i}{j}{k}"))

        for l in range(math.ceil(math.log2(len(S)))):
            
            id += 1
            h[f"l{id}"] = 0
            vals.append((2**l * BigConstant, f"l{id}"))

        square(vals)

Теперь надо привести матрицу к верхнетреугольному виду

In [265]:
print(len(h), len(J))

for k1 in h.keys():
    for k2 in h.keys():
        if k1 != k2 and (k1, k2) in J.keys() and (k2, k1) in J.keys():
            J[(k1, k2)] += J[(k2, k1)]
            del J[(k2, k1)]

print(len(h), len(J))
print(h)
print(J)

154 4528
154 2264
{'x000': 1000000, 'x010': -79999999995243, 'x020': -79999999995866, 'x030': -79999999995488, 'x040': -79999999997255, 'x050': -79999999999697, 'x100': -79999999995243, 'x110': 1000000, 'x120': -79999999997334, 'x130': -79999999996880, 'x140': -79999999998897, 'x150': -79999999999840, 'x200': -79999999995866, 'x210': -79999999997334, 'x220': 1000000, 'x230': -79999999999613, 'x240': -79999999996015, 'x250': -79999999997416, 'x300': -79999999995488, 'x310': -79999999996880, 'x320': -79999999999613, 'x330': 1000000, 'x340': -79999999995732, 'x350': -79999999997870, 'x400': -79999999997255, 'x410': -79999999998897, 'x420': -79999999996015, 'x430': -79999999995732, 'x440': 1000000, 'x450': -79999999997739, 'x500': -79999999999697, 'x510': -79999999999840, 'x520': -79999999997416, 'x530': -79999999997870, 'x540': -79999999997739, 'x550': 1000000, 'u00': -999999998787, 'u10': -999999999239, 'u20': -999999995917, 'u30': -999999995993, 'u40': -999999997332, 'u50': -99999999691

Запустим квантовый отжиг

In [269]:
bqm = BinaryQuadraticModel(h, J, 'BINARY')

min_energy = 0
answer = {}

# Exact solver


# solution = ExactSolver().sample(bqm)
# print("ANSWER:", solution.first.energy)
# print(solution.first.sample)

# min_energy = solution.first.energy
# answer = solution.first.sample


# End Exact solver

for _ in range(1000):
    sample = SimulatedAnnealingSampler().sample(bqm, num_reads=1)
    if sample.first.energy < min_energy:
        min_energy = sample.first.energy
        answer = sample.first.sample

print("Min energy:", min_energy)
print("Optimizing parameters amount:", len(h.keys()))

total_distance = 0

for k in range(CarsNum):

    for i in range(ClientsNum):
        if answer[f"u{i}{k}"] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: Depot -> {i}, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")

    for i in range(ClientsNum):
        for j in range(ClientsNum):
            if answer[f"x{i}{j}{k}"] == 1:
                total_distance += Distances[i][j]
                print(f"Car {k}: {i} -> {j}, distance: {Distances[i][j]}")

    for i in range(ClientsNum):
        if answer[f"n{i}{k}"] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: {i} -> Depot, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")


print("Answer: ", total_distance)

Min energy: -364999999990118.0
Optimizing parameters amount: 154
Car 0: Depot -> 0, distance: 1213
Car 0: 0 -> 5, distance: 303
Car 0: 2 -> 4, distance: 3985
Car 0: 3 -> 2, distance: 387
Car 0: 4 -> 1, distance: 1103
Car 0: 5 -> 3, distance: 2130
Car 0: 1 -> Depot, distance: 761
Answer:  9882


Давайте вычислим ответ полным перебором

In [267]:
answer_complete_search = BigConstant

for perm in itertools.permutations(list(range(ClientsNum))):
    dist = Distances[perm[0]][ClientsNum] + Distances[perm[-1]][ClientsNum]
    for i in range(len(perm) - 1):
        dist += Distances[perm[i]][perm[i + 1]]

    answer_complete_search = min(answer_complete_search, dist)

print("Answer Complete Search:", answer_complete_search)

Answer Complete Search: 9882


Решим задачу используя QDeepSDK

In [ ]:
from qdeepsdk import QDeepHybridSolver
import numpy as np

solver = QDeepHybridSolver()
solver.token = "hf6si03meu"
solver.m_budget = 50000
solver.num_reads = 10000

matrix = np.array([
    [-2, 1, 0, 0 ,1],
    [1, -3, 1, 0, 1],
    [0, 1, -2, 1, 0],
    [0, 0, 1, -2, 1],
    [1, 1, 0, 1, -3]
])

resp = solver.solve(matrix)
res = resp['QdeepHybridSolver']

print(res)